In [ ]:
import requests
from bs4 import BeautifulSoup
import csv 
import time
from tqdm import tqdm
import pandas as pd

In [ ]:
### ruft die api auf und generiert damit eine Liste an links zu allen seiten der mdb
offset= 0
while True:
 
    html = requests.get('https://www.bundestag.de/ajax/filterlist/de/abgeordnete/862712-862712?limit=12&noFilterSet=true&offset='+str(offset)).text
    offset += 12
    soup = BeautifulSoup(html,"html.parser")
    links_zu_mdb = []
    alle_divs = soup.findAll('div', class_='bt-slide-content')
    for div in alle_divs:
        link = div.find('a')['href']
        link = 'https://www.bundestag.de' + link
        links_zu_mdb.append(link)
        print(link)
    with open ('links_zu_allen_mdb.csv',"a") as file:
        writer = csv.writer(file)
        for p in links_zu_mdb:
            writer.writerow([p])
    if 'Leider keine Ergebnisse gefunden' in soup.find('h3').text:
        break
    time.sleep(2)

In [ ]:
## geht durch die liste an links zu den mdb durch und sucht sich deren partei und link zu ihren sm profilen raus
liste_daten_aller_mdb =[]
with open('links_zu_allen_mdb.csv','r') as file:
    data = csv.reader(file)
    for i in tqdm(data):  
        link_zu_mdb =  (i[0])
        liste_daten_des_mdb =[]
        fenster = ''
        partei = 'no_data'
        try:
            html = requests.get(link_zu_mdb).text
            soup = BeautifulSoup(html,"html.parser")
            fenster = soup.find('div',class_='bt-profil row')
            profile = fenster.find('div',class_= 'col-xs-12 col-md-4')
            if profile:
                bt_link_liste = profile.find('ul',class_= 'bt-linkliste')
                if bt_link_liste:
                    links = bt_link_liste.find_all('a')
                    link_liste =[]
                    for link in links:
                        link_dic = {'title': link['title'], 'url': link['href']}
                        link_liste.append(link_dic)
                    h3 = fenster.find('h3').text
                    partei = (h3.split(','))[1]
                    partei = (partei.split('\n'))[0]
                else:
                    link_liste = 'no_data'
            else:
                link_liste = 'no_data'
        except:
            link_liste = 'no_data'
            partei = 'error'
        liste_daten_aller_mdb.append([link_zu_mdb,partei,link_liste])
        liste_daten_des_mdb.append([link_zu_mdb,partei,link_liste])
        with open('daten_aller_abgeordneten_27_06_zwei.csv',"a",newline='')as file:
            writer = csv.writer(file)
            writer.writerow([liste_daten_des_mdb])
        time.sleep(1)


In [ ]:
## speichert die daten in einem besseren format ab
df = pd.DataFrame(liste_daten_aller_mdb,columns = ['mdb','partei','link'])
df.to_csv('daten_aller_abgeordneten_27_06_drei.csv',index=None)